In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

matplotlib.rcParams.update({'figure.figsize': (15, 9)})
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams.update({'axes.labelsize': 20})
matplotlib.rcParams.update({'xtick.labelsize': 12})
matplotlib.rcParams.update({'ytick.labelsize': 12})
matplotlib.rcParams.update({'font.family': 'Helvetica, Arial, sans-serif'})
%config InlineBackend.figure_format = 'retina'

sns.set_style("whitegrid")

# Machine Learning lab 4: extending logistic regression
## Jake Rowland and Paul Herz
2017-10-01

## 1. Introduction

TODO

### 1.1 Background

TODO

### 1.2 Business Case

TODO

### 1.3 Serviceability

TODO

## 2. The dataset: preprocessing and review

### 2.1 Dataset preparation

TODO

### 2.2 Data quality

TODO

## 3. Something



In [2]:
import pandas as pd

m = pd.read_csv('movie_metadata.csv')

# Reorder the DataFrame to a more intelligent fashion
m = m[[
    'movie_title','title_year',
    'genres', 'plot_keywords', 'duration',
    'budget', 'gross',
    'language', 'country', 'content_rating',
    'color', 'aspect_ratio',
    'facenumber_in_poster',
    'director_name',
    'actor_1_name', 'actor_2_name', 'actor_3_name',
    'movie_facebook_likes', 'director_facebook_likes', 'actor_1_facebook_likes', 'actor_2_facebook_likes',
    'actor_3_facebook_likes', 'cast_total_facebook_likes',
    'movie_imdb_link', 'num_user_for_reviews', 'num_critic_for_reviews', 'num_voted_users',
    'imdb_score',
]]

# Reduce the number of float64 data types for columns that do not need a float64 data type
for col in ['title_year','facenumber_in_poster',
'movie_facebook_likes','actor_1_facebook_likes','actor_2_facebook_likes',
'actor_3_facebook_likes','cast_total_facebook_likes','num_user_for_reviews',
'num_critic_for_reviews','num_voted_users']:
    m[col] = pd.to_numeric(m[col],downcast='integer')
  
# Remove all duplicate entries
m.drop_duplicates(inplace=True)

# Create a copy to perserve the original DataFrame
m_original=m.copy()
m.head()

,movie_title,title_year,genres,plot_keywords,duration,budget,gross,language,country,content_rating,...,director_facebook_likes,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,cast_total_facebook_likes,movie_imdb_link,num_user_for_reviews,num_critic_for_reviews,num_voted_users,imdb_score
0,Avatar,2009.0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,178.0,237000000.0,760505847.0,English,USA,PG-13,...,0.0,1000.0,936.0,855.0,4834,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,723.0,886204,7.9
1,Pirates of the Caribbean: At World's End,2007.0,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,169.0,300000000.0,309404152.0,English,USA,PG-13,...,563.0,40000.0,5000.0,1000.0,48350,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,302.0,471220,7.1
2,Spectre,2015.0,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,148.0,245000000.0,200074175.0,English,UK,PG-13,...,0.0,11000.0,393.0,161.0,11700,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,602.0,275868,6.8
3,The Dark Knight Rises,2012.0,Action|Thriller,deception|imprisonment|lawlessness|police offi...,164.0,250000000.0,448130642.0,English,USA,PG-13,...,22000.0,27000.0,23000.0,23000.0,106759,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,813.0,1144337,8.5
4,Star Wars: Episode VII - The Force Awakens ...,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,131.0,131.0,12.0,NaN,143,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,8,7.1


In [3]:
# Remove foreign films to solve the normalization problem
m = m[m['country'] == 'USA']

# Remove items with non-American or non-film rating systems
m = m[m['content_rating'].isin(['R','PG-13','PG','G'])]

In [4]:
# Categorize the IMDB score into three classes:
# [0%-49%] is Poor, [50%-89%] is Average, [90%-100%] is Good.
poor_avg = m['imdb_score'].quantile(.5)
avg_good = m['imdb_score'].quantile(.9)
m['rating_category'] = pd.cut(m.imdb_score,[0,poor_avg,avg_good,10],labels=['poor','average','good'])

In [5]:
for c in ['movie_title','plot_keywords','actor_1_name','actor_2_name','actor_3_name',
'movie_imdb_link','genres', 'director_name','imdb_score','aspect_ratio','country','language']:
    m.drop(c, axis=1, inplace=True)

In [6]:
# Convert categorical values to category type
for col in ['content_rating','color']:
    m[col] = m[col].astype('category')

In [7]:
# Remove rows where the value is null
for col in ['title_year', 'language','country','content_rating',
'aspect_ratio','duration', 'color','gross','budget','movie_facebook_likes',
'actor_1_facebook_likes','actor_2_facebook_likes','actor_3_facebook_likes',
'cast_total_facebook_likes']:
    try:
        m = m[pd.notnull(m[col])]
    except KeyError:
        pass

In [8]:
# Assume null review counts are 0
for col in ['num_user_for_reviews','num_critic_for_reviews']:
    m[col].fillna(value=0,inplace=True)

In [9]:
# Assume missing face counts are the mean
avgFace = round(m['facenumber_in_poster'].mean())
m['facenumber_in_poster'].fillna(value=avgFace, inplace=True)

In [10]:
# replace color and content_rating with dummies
m = pd.get_dummies(m, columns=['color','content_rating'])
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2963 entries, 0 to 5042
Data columns (total 21 columns):
title_year                   2963 non-null float64
duration                     2963 non-null float64
budget                       2963 non-null float64
gross                        2963 non-null float64
facenumber_in_poster         2963 non-null float64
movie_facebook_likes         2963 non-null int32
director_facebook_likes      2963 non-null float64
actor_1_facebook_likes       2963 non-null float64
actor_2_facebook_likes       2963 non-null float64
actor_3_facebook_likes       2963 non-null float64
cast_total_facebook_likes    2963 non-null int32
num_user_for_reviews         2963 non-null float64
num_critic_for_reviews       2963 non-null float64
num_voted_users              2963 non-null int32
rating_category              2963 non-null category
color_ Black and White       2963 non-null uint8
color_Color                  2963 non-null uint8
content_rating_G             2963 n

In [11]:
if m.rating_category.dtype != np.dtype('int8'):
    m.rating_category = m.rating_category.cat.codes
m.rating_category.value_counts()

0    1456
1    1231
2     276
Name: rating_category, dtype: int64

In [12]:
X=m.drop('rating_category', axis=1, inplace=False)
y=np.ravel(m['rating_category'])

split_index = int(len(m)*0.8)
X_train = X[:split_index]
X_predict = X[split_index:]
y_train = y[:split_index]
y_predict = y[split_index:]

type(y_train)

numpy.ndarray

In [13]:
logistic = linear_model.LogisticRegression(multi_class='ovr')
logistic.fit(X_train,y_train)
C = logistic.predict(X_predict)
print('Predicted', pd.Series(C).value_counts())
print('Actual', pd.Series(y_predict).value_counts())

from sklearn.metrics import confusion_matrix



m = confusion_matrix(C,y_predict)
print(m)

logistic.coef_

#  Default
# [[130 156  38]
#  [ 85  98  10]
#  [ 43  31   2]]

# Normalized
#[[130 156  38]
# [ 85  98  10]
# [ 43  31   2]]

Predicted 1    369
0    200
2     24
dtype: int64
Actual 1    285
0    258
2     50
dtype: int64
[[139  60   1]
 [119 220  30]
 [  0   5  19]]


array([[  2.70670632e-06,   9.29206545e-08,   1.94725569e-08,
          4.26882976e-09,   3.03963961e-09,  -7.88845810e-06,
         -1.67991398e-06,   1.22319154e-06,   2.05359998e-07,
          7.82966274e-07,   3.77450250e-06,   1.93695656e-07,
          7.69562936e-08,  -1.52688185e-05,  -6.03054113e-11,
          1.40916377e-09,  -5.27423771e-11,  -7.76609739e-11,
          1.10956127e-09,   3.69700441e-10],
       [ -2.87471915e-06,  -1.04729003e-07,  -6.69589930e-09,
          2.53262363e-09,  -2.91174758e-09,   5.04803939e-06,
          2.30343391e-07,   3.44911792e-06,   3.21321326e-07,
         -6.45138284e-07,   1.41261683e-06,  -2.55820800e-08,
          5.51190945e-08,  -1.04373501e-06,   4.22430351e-11,
         -1.47854413e-09,  -4.18235734e-11,  -2.87056491e-10,
         -1.02639122e-09,  -8.10298029e-11],
       [ -1.43006905e-03,  -6.47127613e-05,  -1.86114605e-08,
         -4.26360138e-09,  -1.24093552e-06,   7.44396755e-06,
          3.44411828e-05,   8.24067608e-05

In [24]:
from scipy.special import expit

class Regression:

    def __init__(self, eta, maxIter=20, C=0.001, opt='GD', reg ='NA'):
        self.eta = eta
        self.maxIter = maxIter
        self.C = C
        self.optimization = opt
        self.regularization = reg
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        # ydiff = y(m,1) - yHat(m,1)
        # The ravel of ydiff is 1-D array of size m
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        # X is (m,n+1)
        # ydiff is 1-D array of size m. ydiff[:,np.newaxis] is matrix of (m,1)
        # X * ydiff[:,np.newaxis] is matrix (m,n+1) where each row is X[m] @ ydiff
        # Calculate the horizontal average for each row. Result is 1-D array of size m
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        # Reshape the gradient to be matrix (m,1)
        gradient = gradient.reshape(self.w_.shape)
        
        if(self.regularization == 'L1'):
            # L1 = C * sum(|w|)
            gradient[1:] += self.C * np.sum(abs(self.w_[1:])) 
        elif(self.regularization == 'L2'):
            # L2 = C * sum(|w|^2)
            gradient[1:] += self.w_[1:]*self.w_[1:] * self.C
            #gradient[1:] += -2 * self.w_[1:] * self.C
        elif(self.regularization == 'L1/L2'):
            # DOES NOT WORK
            #Calculate the L1 and L2 regularization
            L1 = np.sum(abs(self.w_[1:]))
            L2 = self.w_[1:]*self.w_[1:]
            # Mutlipy the quotient of L1 and L2 by regularization rate
            gradient[1:] += np.divide(L1,L2) * self.C
        
        return gradient
    
    def _gradient_descent(self):
        # for as many as the max iterations
        for _ in range(self.maxIter):
            # Get the gradient of form (m,1)
            gradient = self._get_gradient(self.Xb,self.y)
            # Add the associated gradient to the weight to adjust the weights.
            self.w_ += gradient*self.eta # multiply by learning rate
            
            
    def _stocastic_descent(self):
        for _ in range(self.maxIter):
            for m in range(1,self.num_samples):
                # Calulate the weights for 1st instance, then 2nd, then ..., then mth
                gradient = self._get_gradient(self.Xb[:m], self.y[:m])
                self.w_ += gradient*self.eta # multipy by learning rate
    
            
    def _newton_H(self,X,y):
        # Get P(y=1|x,w_)
        hX = self.predict_proba(X,add_bias=False)
        # get P(y=0|x,w_)
        one_hX = 1 - hX
        
        # Multiply together
        prob_Mult = hX*one_hX
        
        # Create matrix to perform average on
        H = np.zeros((X.shape[1], X.shape[1]))
        for row in range(X.shape[0]):
            xi = X[row] # Get the row of X
            xi.shape = (X.shape[1],1) #Convert to 2D array
            # Get the transpose of the row
            xiT = xi.T
            
            # Create a matrix from x and xT
            xMat = xi*xiT
            
            # Scalar multipy the probibility multiplyer 
            # and the xMatrix
            xMat = prob_Mult[row] * xMat
            
            # Add the matrix to accumulation matrix
            H = H + xMat
            
        # Divide the accumulation matrix by the number of samples to get average
        H = H / X.shape[0]
        
        return H
    
        
    def _newton_method(self):
        for _ in range(self.maxIter):
            # Get the hessian matrix (Second derivative)
            H = self._newton_H(self.Xb, self.y)
            # Get the gradient (First Derivative)
            gradient = self._get_gradient(self.Xb, self.y)
            try:
                # Attempt to inverse the hessian
                H_inv = np.linalg.inv(H)
            
                # Modify the w_(t+1) = w_(t) - (dJ(w_))/(d^2J(w_))
                self.w_ -= (H_inv @ gradient)
            except:
                print('Singularity Matrix')
    
    # public:
    # Takes (m,n) matrix - X. Add bias term to X matrix to create (m,n+1) matrix - Xb
    # Calculate the dot product of Xb(m,n+1) and w_(n+1,1) -> result is matrix of (m,1)
    # Calculate the sigmoid for each value of the dot product and return a matrix of (m,1)
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X,prob=.5):
        return (self.predict_proba(X) >= prob)
    
    def predict_raw(self,X):
        return (self.predict_proba(X))
        
    def fit(self, X, y):
        self.Xb = self._add_bias(X) # add bias term
        self.num_samples, self.num_features = self.Xb.shape
        self.y = y
        
        self.w_ = np.zeros((self.num_features,1)) # init weight vector to zeros
        
        # Select the optimization process
        if(self.optimization == 'GD'):
            self._gradient_descent()
        elif (self.optimization == 'SGD'):
            self._stocastic_descent()
        elif (self.optimization == 'NWT'):
            self._newton_method()
        else:
            self._gradient_descent()

In [25]:
class MultiClassRegression:
    
    def __init__(self, eta, maxIter=20, C=0.001, opt='GD', reg ='NA'):
        self.eta = eta
        self.maxIter = maxIter
        self.C = C
        self.optimization = opt
        self.regularization = reg
        
    def fit(self, X, y):
        self.X = X
        self.true_y = y
        
        # Find unique classes
        self.classes = np.unique(y).tolist()
        
        #Create dictonary to hold regression class associated with the class as the key
        self.reg = {}
        
        # Define training classes as matrix of (n_class, n_samples)
        self.y_classes = np.zeros((len(self.classes),X.shape[0]))
        # Define weights as matrix of (n_class, n_features)
        self.w_ = np.zeros((len(self.classes),(X.shape[1]+1)))
        
        # For every class
        for cur_class in self.classes:
            # Get the indexes of all classes that are positive
            idx = y == cur_class
            
            # Define the positive classes as 1 in a zeroed array
            self.y_classes[cur_class][idx] = 1
            
            # Find the index of the class (for non numerical classes)
            idx_class = self.classes.index(cur_class)
            
            # Create a binary logistic classifier with the parameters passed to the multiclass logistic classifier
            self.reg[idx_class] = Regression(self.eta, self.maxIter, self.C, self.optimization, self.regularization)
            # Fit all the binary logistic classifiers with the training data and their associated training classes
            self.reg[idx_class].fit(X,self.y_classes[cur_class])
            
            # Copy weights from the binary to the associated location in the multiclass logistic classifer
            self.w_[idx_class] = self.reg[idx_class].w_.ravel()
            
            
    def predict(self, X):
        # Create a prediction matrix of size (n_classes, n_samples)
        prediction = np.zeros((len(self.classes),len(X)))
        
        # For each binary logistic classifire
        for key, reg in self.reg.items():
            # Get index of key (for non numerical classes)
            idx = self.classes.index(key)
            # Predict the raw probability for each sample and make it a 1D array
            prediction[key] = reg.predict_raw(X).ravel()
            
        # Return the row index that has the maximum prediction value
        return np.argmax(prediction, axis = 0)
    
    def predict_raw(self, X):
        # Create a prediction matrix of size (n_classes, n_samples)
        prediction = np.zeros((len(self.classes),len(X)))
        
        # For each binary logistic classifire
        for key, reg in self.reg.items():
            # Get index of key (for non numerical classes)
            idx = self.classes.index(key)
            # Predict the raw probability for each sample and make it a 1D array
            prediction[key] = reg.predict_raw(X).ravel()
            
        # Return raw prediciton values for each class against each sample
        return prediction
        

In [26]:
#Do not remove. This is our best estimation so far
# multiR = MultiClassRegression(eta=.01, maxIter=5, C=.1, opt='NWT', reg='L1')
multiR = MultiClassRegression(eta=.01, maxIter=5, C=.01, opt='NWT', reg='L1')

In [27]:
multiR.fit(X_train, y_train)
pred = multiR.predict(X_predict)

histo = np.bincount(pred)
print(maxes)
print(histo)

m = confusion_matrix(maxes,y_predict)
print(m)


print(multiR.w_)

Singularity Matrix
Singularity Matrix
Singularity Matrix
Singularity Matrix
Singularity Matrix
Singularity Matrix
Singularity Matrix
Singularity Matrix
Singularity Matrix
[2 0 0 0 0 1 1 0 0 2 0 0 0 0 0 0 0 1 0 1 0 0 1 2 0 1 1 1 0 1 1 0 1 1 0 1 0
 1 0 1 0 0 1 1 0 1 0 2 0 0 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 2
 0 1 0 0 0 2 0 0 0 0 0 1 1 0 1 2 2 2 1 0 2 2 2 2 0 0 1 0 1 0 0 0 0 1 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 2 1 0 1 0 2 1 0 1 0 2 0 0 2 2 0 1
 2 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 0 2 1 0 1 2 0 1 0 1 1 1 0 0 0 1 0 1
 2 1 0 0 0 0 0 0 0 0 0 0 0 1 1 2 0 1 0 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 0
 1 1 1 1 2 0 1 0 0 0 0 0 2 0 2 0 1 0 2 1 1 1 0 0 1 0 2 1 0 0 0 0 1 0 1 0 0
 0 1 1 0 1 1 1 1 1 0 2 1 1 0 2 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 2 0 0 0 1 0 0 0 0 0 0 0 1 1 2 0 2 2 2 0 2 0 0
 2 2 1 2 0 0 0 1 0 0 0 0 0 0 2 0 0 2 0 1 0 0 1 2 0 1 1 2 1 0 0 2 0 0 2 0 2
 0 0 0 2 1 1 0 1 0 0 0 0 1 2 0 0 0 2 1 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1
 0 0

In [18]:
#Normalize the data to better fit a gradient curve
X_predict_norm = X_predict
X_train_norm = X_train
for col in X_train_norm.columns.values:
    mx = X_train_norm[col].max()
    X_train_norm[col] = X_train_norm[col]/mx
    X_predict_norm[col] = X_predict_norm[col]/mx
    
#NOT USED CURRENTLY

/Users/jake.rowland/.local/share/virtualenvs/Project4-JwQJWw-L/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jake.rowland/.local/share/virtualenvs/Project4-JwQJWw-L/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Training data      [2 1 2 ..., 0 0 0]
Training 0 targets [0 0 0 ..., 1 1 1]
Training 1 targets [0 1 0 ..., 0 0 0]
Training 2 targets [0 0 0 ..., 0 0 0]
